In [1]:
import psycopg2
import numpy as np

print('connecting to postgres...')
connection = psycopg2.connect(user="max",
                              password="password",
                              host="127.0.0.1",
                              port="5432",
                              database="final")
print()
print(connection)
cursor = connection.cursor()

connecting to postgres...

<connection object at 0x7fc7a9fa6768; dsn: 'user=max password=xxx dbname=final host=127.0.0.1 port=5432', closed: 0>


In [26]:
q = """
select * from spotify_features;
"""

cursor.execute(q)

In [27]:
spotify = cursor.fetchall()

In [28]:
print(spotify[500])

('...And You Will Know Us by the Trail of Dead Novena Without Faith', '...and you will know us by the trail of dead novena without faith', Decimal('0.155'), Decimal('0.568'), Decimal('2'), Decimal('-10.029'), Decimal('1'), Decimal('0.0372'), Decimal('0.00201'), Decimal('0.885'), Decimal('0.128'), Decimal('0.0398'), Decimal('168.419'), 'spotify:track:1yvp6AHRwXifM9pDAIrRwD', 'https://api.spotify.com/v1/audio-analysis/1yvp6AHRwXifM9pDAIrRwD', Decimal('503973'), 4)


so that's pretty messy. Let's load it up into a dataframe

In [34]:
import pandas as pd
import numpy as np

columns = [
    'lastfm_artist_track',
    'spotify_artist_track',
    'danceability',
    'energy',
    'key',
    'loudness',
    'mode', 
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
    'uri',
    'analysis_url', 
    'duration_ms',
    'time_signature' 
]

df = pd.DataFrame(spotify, columns=columns)

# want to convert SQL Decimal(val) to float(val)
feats = [
    'danceability',
    'energy',
    'key',
    'loudness',
    'mode', 
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo', 
    'duration_ms',
    'time_signature' 
]

df[feats] = df[feats].astype(float)


df.head()

,lastfm_artist_track,spotify_artist_track,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,analysis_url,duration_ms,time_signature
0,(Sandy) Alex G String,(sandy) alex g string,0.636,0.567,9.0,-6.003,1.0,0.0362,0.0593,0.0484,0.0815,0.370,105.625,spotify:track:5QUcQ7OmRTzGAZT6IkhMO8,https://api.spotify.com/v1/audio-analysis/5QUc...,294867.0,4.0
1,(Sandy) Alex G Sum Secret,(sandy) alex g sum secret,0.654,0.502,3.0,-11.275,1.0,0.0270,0.7520,0.7310,0.3820,0.674,91.950,spotify:track:6GC4JLu0WBhhrInCdGNlCq,https://api.spotify.com/v1/audio-analysis/6GC4...,219783.0,4.0
2,(Sandy) Alex G Thorns,(sandy) alex g thorns,0.569,0.306,2.0,-14.128,1.0,0.0363,0.3980,0.0195,0.1410,0.492,125.923,spotify:track:7xMWwvpUjihrid15USMtlw,https://api.spotify.com/v1/audio-analysis/7xMW...,170108.0,3.0
3,(Sandy) Alex G Trick,(sandy) alex g mary,0.693,0.467,4.0,-9.558,0.0,0.0430,0.0497,0.4520,0.1070,0.402,105.958,spotify:track:4q8CkKKnXQxiH4bTVKCm1I,https://api.spotify.com/v1/audio-analysis/4q8C...,197733.0,4.0
4,(Sandy) Alex G Tripper,(sandy) alex g tripper,0.717,0.297,0.0,-13.305,1.0,0.0447,0.9920,0.8920,0.1260,0.653,119.875,spotify:track:7Bc55L9HYcBBulGlSLAoR0,https://api.spotify.com/v1/audio-analysis/7Bc5...,42133.0,5.0


In [30]:
X = df[feats]
names = df.spotify_artist_track

### baseline model: distance

In [31]:
# start by standard scalar
from sklearn.preprocessing import minmax_scale

X_scaled = minmax_scale(X, feature_range=(0, 10))

In [33]:
len(X)

1000